In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import DBSCAN, AgglomerativeClustering
from sklearn.metrics import silhouette_score, davies_bouldin_score
from scipy.stats import zscore
import hdbscan
from itertools import product

In [2]:
df = pd.read_csv("C:/Users/gabri/OneDrive/Documents/Faculdade/2024.2/Projeto de Ciência de Dados/novo/volleyball-case-study/data/para-o-projeto/team_v_team.csv")
df.head()

,Unnamed: 0,Date,TeamA,TeamB,Result,S,Team A Kills,Team A Errors,Team A Total Attacks,Team A Hit Pct,...,Team B Hit Pct,Team B Assists,Team B Aces,Team B SErr,Team B Digs,Team B RErr,Team B Block Solos,Team B Block Assists,Team B BErr,Team B PTS
0,0,8302019,USC Upstate,Virginia Tech,0,4.0,33,21,112,0.107,...,0.234,48,13,12,51,4,4,20,2,0.0
1,1,8302019,USC Upstate,Northwestern,0,3.0,16,24,89,-0.090,...,0.349,41,6,7,42,1,2,20,0,0.0
2,2,9072019,USC Upstate,Ga. Southern,0,3.0,37,20,117,0.145,...,0.239,39,4,5,49,2,1,6,1,50.0
3,3,9132019,USC Upstate,Norfolk St.,1,5.0,50,27,164,0.140,...,0.114,47,11,15,67,9,1,17,1,70.5
4,4,9142019,USC Upstate,George Washington,0,3.0,30,17,112,0.116,...,0.216,44,3,1,60,2,1,6,6,52.0


In [3]:
# Gerando colunas para obter eficiência geral

df["Team A Total Errors"] = df['Team A Errors'] + df['Team A SErr'] + df['Team A BErr'] + df['Team A RErr']
df["Team B Total Errors"] = df['Team B Errors'] + df['Team B SErr'] + df['Team B BErr'] + df['Team B RErr']
df["Team A Total Rounds"] = df['Team A PTS'] + df['Team A Total Errors']
df["Team B Total Rounds"] = df['Team B PTS'] + df['Team B Total Errors']
df["Team A Efficiency Rate"] = df["Team A PTS"] - df["Team A Total Errors"] / df["Team A Total Rounds"]
df["Team B Efficiency Rate"] = df["Team B PTS"] - df["Team B Total Errors"] / df["Team B Total Rounds"]

# Gerando colunas para obter eficiência de bloqueio

df["Team A Total Blocks"] = df['Team A Block Solos'] + df['Team A Block Assists']*0.5 + df['Team A BErr']
df["Team B Total Blocks"] = df['Team B Block Solos'] + df['Team B Block Assists']*0.5 + df['Team B BErr']
df["Team A Success Blocks"]= df['Team A Block Solos'] + df['Team A Block Assists']*0.5
df["Team B Success Blocks"]= df['Team B Block Solos'] + df['Team B Block Assists']*0.5
df["Team A Block Efficiency"] = df["Team A Success Blocks"] / df["Team A Total Blocks"]
df["Team B Block Efficiency"] = df["Team B Success Blocks"] / df["Team B Total Blocks"]

# Gerando colunas para obter a eficiência de defesa

df["Team A Total Digs"] = df['Team A Digs'] + df['Team A RErr']
df["Team B Total Digs"] = df['Team B Digs'] + df['Team B RErr']
df["Team A Efficiency Digs"] = df["Team A Digs"] / df["Team A Total Digs"]
df["Team B Efficiency Digs"] = df["Team B Digs"]  / df["Team B Total Digs"]

ind_remocao = df["Team A Total Errors"].loc[df["Team A Total Errors"] == 0].index
df.drop(ind_remocao, inplace=True)

# ind_remocao = df["Team A Block Efficiency"].loc[df["Team A Block Efficiency"] == 0].index
# df.drop(ind_remocao, inplace=True)
# ind_remocao = df["Team B Block Efficiency"].loc[df["Team B Block Efficiency"] == 0].index
# df.drop(ind_remocao, inplace=True)

ind_remocao = df["Team B Total Blocks"].loc[df["Team B Total Blocks"] == 0].index
df.drop(ind_remocao, inplace=True)
ind_remocao = df["Team A Total Blocks"].loc[df["Team A Total Blocks"] == 0].index
df.drop(ind_remocao, inplace=True)


In [4]:
# Criando um dicionário para mapear os IDs dos times
team_ids = {team: idx for idx, team in enumerate(set(df["TeamA"]).union(df["TeamB"]))}

# Adicionando colunas de IDs de time
df["TeamA_ID"] = df["TeamA"].map(team_ids)
df["TeamB_ID"] = df["TeamB"].map(team_ids)

# Criando a coluna de ID da partida, garantindo que os jogos reversos tenham o mesmo ID
df["Match_ID"] = df.apply(lambda row: (row["Date"], tuple(sorted([row["TeamA_ID"], row["TeamB_ID"]]))), axis=1)

# Convertendo a combinação (data, time A ID, time B ID) em um identificador numérico único
df["Match_ID"] = df["Match_ID"].apply(lambda x: hash(x))


In [5]:
def normaliza(df):
    scaler = MinMaxScaler()
    df_norm = df
    df_norm = scaler.fit_transform(df)
    return df_norm

In [6]:
def outlier(df):
    z_scores = zscore(df, nan_policy='omit')
    df_norm_clean = df[(abs(z_scores) < 3).all(axis=1)]
    return df_norm_clean

In [7]:
colunas_bloq = ['Team A Block Solos', 'Team A Block Assists', 'Team A BErr','Team A Block Efficiency','Team A Total Blocks', 'Team A Success Blocks']
colunas_bloq_fe = ['Team A Block Solos', 'Team A Block Assists', 'Team A BErr']

df_bloq = df[colunas_bloq]
df_bloq_fe = df[colunas_bloq_fe]

lst_df = [df_bloq,df_bloq_fe]

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import DBSCAN, AgglomerativeClustering, KMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score
from scipy.stats import zscore
from itertools import product

# Função para gerar cenários
def generate_scenarios():
    normalizations = [True, False]
    outliers = [True, False]
    dataframes = ['df_bloq', 'df_bloq_fe']
    models = ['dbscan', 'kmeans', 'hierarchical']
    scenarios = list(product(normalizations, outliers, dataframes, models))
    return scenarios

# Função para aplicar modelo
def apply_model(model, data, params):
    if model == 'dbscan':
        cluster = DBSCAN(
            eps=params.get('eps', 0.5),
            min_samples=params.get('min_samples', 5),
            metric=params.get('metric', 'euclidean'),
            algorithm=params.get('algorithm', 'auto')
        )
    elif model == 'kmeans':
        cluster = KMeans(
            n_clusters=params.get('n_clusters', 3),
            init=params.get('init', 'k-means++'),
            max_iter=params.get('max_iter', 300),
            random_state=42
        )
    elif model == 'hierarchical':
        cluster = AgglomerativeClustering(
            n_clusters=params.get('n_clusters', 2),
            linkage=params.get('linkage', 'ward'),
            affinity=params.get('affinity', 'euclidean')
        )
    else:
        raise ValueError(f"Modelo {model} não suportado.")
    labels = cluster.fit_predict(data)
    return labels

# Função para avaliar modelo
def evaluate_model(data, labels):
    if len(np.unique(labels)) > 1:  # Evitar erro de métricas com apenas 1 cluster
        silhouette = silhouette_score(data, labels)
        davies_bouldin = davies_bouldin_score(data, labels)
    else:
        silhouette = None
        davies_bouldin = None
    return silhouette, davies_bouldin

# Função principal
def execute_scenarios(df, output_file='resultados_cenarios.csv'):
    scenarios = generate_scenarios()
    results = []

    # Abrir ou criar o arquivo CSV de resultados
    try:
        existing_results = pd.read_csv(output_file)
        processed_scenarios = set(existing_results['Scenario ID'])
    except FileNotFoundError:
        processed_scenarios = set()
        with open(output_file, 'w') as f:
            f.write('Scenario ID,Normalization,Remove Outliers,DataFrame Type,Model,Params,Silhouette Score,Davies-Bouldin Index\n')

    for scenario_id, scenario in enumerate(scenarios):
        # Identificar o ID do cenário
        if scenario_id in processed_scenarios:
            continue  # Pular cenários já processados

        norm, remove_outliers, dataframe_type, model_type = scenario

        # Selecionar o dataframe
        if dataframe_type == 'df_bloq':
            data = df[colunas_bloq].copy()
        else:
            data = df[colunas_bloq_fe].copy()

        # Aplicar normalização e remoção de outliers
        if norm:
            data = normaliza(data)
        if remove_outliers:
            data = outlier(data)

        # Testar diferentes hiperparâmetros
        if model_type == 'dbscan':
            param_grid = [
                {'eps': eps, 'min_samples': min_samples, 'metric': metric, 'algorithm': algorithm}
                for eps in np.linspace(0.1, 2, 10)
                for min_samples in range(3, 10)
                for metric in ['euclidean', 'manhattan', 'cosine']
                for algorithm in ['auto', 'ball_tree', 'kd_tree', 'brute']
            ]
        elif model_type == 'kmeans':
            param_grid = [
                {'n_clusters': n_clusters, 'init': init, 'max_iter': max_iter}
                for n_clusters in range(2, 10)
                for init in ['k-means++', 'random']
                for max_iter in [100, 300, 500]
            ]
        elif model_type == 'hierarchical':
            param_grid = [
                {'n_clusters': n_clusters, 'linkage': linkage, 'affinity': affinity}
                for n_clusters in range(2, 10)
                for linkage in ['ward', 'complete', 'average', 'single']
                for affinity in ['euclidean', 'manhattan', 'cosine']
            ]
        else:
            continue

        # Iterar sobre combinações de parâmetros
        for params in param_grid:
            try:
                labels = apply_model(model_type, data, params)
                silhouette, davies_bouldin = evaluate_model(data, labels)

                # Criar linha de resultado
                result = {
                    'Scenario ID': scenario_id,
                    'Normalization': norm,
                    'Remove Outliers': remove_outliers,
                    'DataFrame Type': dataframe_type,
                    'Model': model_type,
                    'Params': params,
                    'Silhouette Score': silhouette,
                    'Davies-Bouldin Index': davies_bouldin
                }
                results.append(result)

                # Adicionar ao CSV
                pd.DataFrame([result]).to_csv(output_file, mode='a', index=False, header=False)

            except Exception as e:
                print(f"Erro no cenário {scenario} com parâmetros {params}: {e}")

    # Retornar todos os resultados acumulados como DataFrame
    return pd.DataFrame(results)


In [ ]:
results = execute_scenarios(df)
results.to_csv('resultados_cenarios.csv', index=False)